In [1]:
import pandas as pd

In [3]:
c_met = pd.read_csv("../HA_data/metadata/crystals_metadata.csv", index_col=0)
c_seq = pd.read_csv("../HA_data/aligned_sequences/crystal_aln.csv", index_col=0)

# NOT the same
# I needed to add header: id,chain,host,location,year
m_met = pd.read_csv("../HA_data/metadata/models_metadata.csv", index_col=0, na_values=["-"])
m_seq = pd.read_csv("../HA_data/aligned_sequences/models_aln.csv", index_col=0)
m_template = pd.read_csv("../HA_data/models_template_mapping.csv", index_col=0)

# Crystal data

In [4]:
c_met.head()

,chain,subtype,host,location,year
id,,,,,
1ha0,A,H3N2,NaN,Aichi,1968
4eef,A,H1N1,NaN,BrevigMission,1918
4cqp,A,H5N1,NaN,Vietnam,2004
4cqq,A,H5N1,NaN,Vietnam,2004
4cqr,A,H5N1,NaN,Vietnam,2004


In [5]:
"H3N2".split("\w")

['H3N2']

In [6]:
len(c_met.location.value_counts())

52

In [7]:
c_met.host.value_counts()

avian                     54
human                     14
canine                     7
equine                     7
reassortant X31            6
swine                      5
bat                        2
reassortant IDCDC_RG29     1
dtype: int64

In [8]:
def clean_host(x):
    if pd.isnull(x):
        return "human"
    elif x.find("reassortant") > -1:
        return "reassortant"
    else:
        return x.replace("_", " ")

In [9]:
c_met.host = c_met.host.apply(clean_host)

In [12]:
c_met["H"] = c_met.subtype.apply(lambda x: int(x.split("N")[0][1:]))
c_met["N"] = c_met.subtype.apply(lambda x: int(x.split("N")[1]) if x[-1] != "x" else "")

In [13]:
c_seq.head()

,sequence
p_id,
4bgx,-DQICIGYHANNSTEQVDTIMEKNVTVTHAQDILEKTHNGKLCDLD...
4bgy,-DQICIGYHANNSTEQVDTIMEKNVTVTHAQDILEKTHNGKLCDLD...
4bgw,-DQICIGYHANNSTEQVDTIMEKNVTVTHAQDILEKTHNGKLCDLD...
3znm,-DQICIGYHANNSTEQVDTIMEKNVTVTHAQDILEKTHNGKLCDLD...
3znl,-DQICIGYHANNSTEQVDTIMEKNVTVTHAQDILEKTHNGKLCDLD...


In [14]:
# sequence for other data missing (but not the other way)
c_seq.index.difference(c_met.index)

Index([], dtype='object')

In [15]:
c_all = c_met.join(c_seq, how="outer")

In [16]:
c_all.head()

,chain,subtype,host,location,year,H,N,sequence
1eo8,A,H3N2,human,Aichi,1968,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN-...
1ha0,A,H3N2,human,Aichi,1968,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN-...
1hgd,A,H3N2,reassortant,NaN,NaN,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN-...
1hge,A,H3N2,reassortant,NaN,NaN,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN-...
1hgf,A,H3N2,reassortant,NaN,NaN,3,2,TATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNN-...


In [27]:
c_all.index.name = "id"

In [28]:
c_all.to_csv("../site/data/crystals_metadata.csv", float_format = '%.0f')  # for missing H/N

## Model data

In [18]:
m_met.head()

,chain,host,location,year,subtype
id,,,,,
AAA43090,A,NaN,Berlin,1964,H2N2
AAA43117,A,avian,NaN,1972,H2N9
AAA43185,A,NaN,Japan,1957,H2N2
AAA43205,A,avian,Ontario,1966,H5N9
AAA43243,A,avian,Hong Kong,1978,H2N2


In [19]:
m_met.host.value_counts()

swine             485
avian             326
emu                 2
ferret              1
giant anteater      1
dtype: int64

In [20]:
m_met.host = m_met.host.fillna("human")

In [21]:
len(m_met.location.value_counts())

316

In [22]:
m_seq.head()

,sequence
p_id,
AAA43090,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
AAA43247,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
ABO38701,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
ABO44057,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
ABO52247,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...


In [23]:
m_template.head()

,template_id
p_id,
AAA43578,2wr5
AEK49568,4wsw
AGL58994,4wsw
AGL60617,4wsw
AGL60775,4wsw


In [24]:
m_all = m_met.join(m_template, how="outer")
m_all = m_all.join(m_seq, how="outer")
m_all.index.name = "p_id"

In [25]:
m_all.head()

,chain,host,location,year,subtype,template_id,sequence
p_id,,,,,,,
AAA43090,A,human,Berlin,1964,H2N2,3ku6,GDQICIGYHAN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
AAA43117,A,avian,NaN,1972,H2N9,3ku6,GDQICIGYHSN-STEKVDTILERNVTVTHAKDILEK--THNGKLCK...
AAA43185,A,human,Japan,1957,H2N2,3ku6,GDQICIGYHAN-STEKVDTNLERNVTVTHAKDILEK--THNGKLCK...
AAA43205,A,avian,Ontario,1966,H5N9,1jso,-DQICIGYHAN-STKQVDTIMEKNVTVTYAQDILEK--EHNGKLCS...
AAA43243,A,avian,Hong Kong,1978,H2N2,3ku6,GDQICIGYHAN-STETVDTILERNVTVTHAKNILEK--THNGKLCK...


In [26]:
# WARNING: as of now without serotype or subtype
m_all.to_csv("../site/data/models_metadata.csv", float_format = '%.0f')